In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import _pickle as pickle

## Some useful functions to ease the processings
def save(file,name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'wb')
    else:
        outfile = open(name+'.pickle', 'wb')
    pickle.dump(file, outfile)
    outfile.close
    
def load(name, folder = ""):
    if folder != "":
        outfile = open('./'+folder+'/'+name+'.pickle', 'rb')
    else:
        outfile = open(name+'.pickle', 'rb')
    file = pickle.load(outfile)
    outfile.close
    return file

from tqdm.notebook import tqdm
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.preprocessing.text import Tokenizer

import os

In [ ]:
f = open('fr_dedup.txt', 'r', encoding = 'utf-8')

In [ ]:
size_batch = 500000


for i, line in enumerate(f):
    
    if i % size_batch == 0:
        if i != 0:
            save(batch, 'batch_'+str(i//size_batch), 'batch')
        batch = list(np.zeros(size_batch))
        
    batch[i%size_batch] = line
    
    if i == 40000000:
        break
    

In [ ]:
f.close()

In [ ]:
test_file = load('batch_1', 'batch')

In [ ]:
def clean(x):
    import unidecode
    x = str(x)
    x = x.lower()
    x = " ".join(x.split())
    x = unidecode.unidecode(x)
    x = x.replace('\n', '')
    x = '[CLS] ' + x + ' [SEP]'
    return x

In [ ]:
from multiprocess import Pool

p = Pool(8)
x = p.map(clean, test_file)
p.close()

In [ ]:

tokenizer = Tokenizer(num_words = 30000, lower = False, filters = '', oov_token = '[UNK]')


x0 = []
for i in range(20000):
    x1 = " ".join("[CLS]" for elt in range(60))
    x2 = " ".join("[SEP]" for elt in range(50))
    x0.append(x1)
    x0.append(x2)
tokenizer.fit_on_texts(x0 + x)

tokenizer.word_index['[PAD]'] = 0
tokenizer.index_word[0] = '[PAD]'
tokenizer.word_counts['[PAD]'] = 30000
tokenizer.word_counts['[UNK]'] = 30000
wi = {}
wc = {}
iw = {}
for key in tokenizer.word_index.keys():
    if tokenizer.word_counts[key] >= 40:
        wi[key] = tokenizer.word_index[key]
        iw[tokenizer.word_index[key]] = key
        wc[key] = tokenizer.word_counts[key]
tokenizer.word_index = wi
tokenizer.index_word = iw
tokenizer.word_counts = wc

In [ ]:
save(tokenizer, 'IFart_tokenizer')

In [ ]:
tokenizer = load('IFart_tokenizer')

for i in tqdm(range(1,81)):
    test_file = load('batch_'+str(i), 'batch')
    
    p = Pool(8)
    file_clean = p.map(clean, test_file)
    p.close()
    
    save(file_clean,'batch_'+str(i), 'batch_clean')
    
    tok = tokenizer.texts_to_sequences(file_clean)
    
    pad = pad_sequences(tok, maxlen=512, dtype='int32', padding='post', truncating='post',
    value=0.0)
    
    pad[pad[:,-1] != 0, -1] = 2
    
    save(pad,'batch_'+str(i), 'batch_tokenized')

## Building Model

In [2]:
from tf_transformers2 import *
from tensorflow.keras.layers import Input, Dense, Dropout, TimeDistributed, LSTM
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
tokenizer = load('IFart_tokenizer')

inputs_ids = Input(shape = (None,))
# inputs_types = Input(shape = (None,))

vocab_size = len(tokenizer.word_index)

# inputs = [inputs_ids, inputs_types]

inputs = inputs_ids

# GPT = GPTDecoder(num_layers = 12, d_model = 768, num_heads = 12, dff = 3072, target_vocab_size = vocab_size,
#                maximum_position_encoding = 512, rate=0.1, bidirectional_decoder = False)

GPT = GPTDecoder(num_layers = 12, d_model = 512, num_heads = 8, dff = 1024, target_vocab_size = vocab_size,
               maximum_position_encoding = 512, rate=0.1, bidirectional_decoder = False)

outputs, _ = GPT(inputs_ids, training = True, token_types_ids = None)

outputs = tf.keras.layers.Dense(vocab_size)(outputs)

model = Model(inputs, outputs)

In [4]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000, factor = 1):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
        self.factor = factor
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2) / self.factor
    
def sparse_k_acc(a,b):
    
    if a.shape[1] is None:
        return 1
    else:
        a = tf.split(a, a.shape[1], axis = 1)
        b = tf.split(b, b.shape[1],axis = 1)
        c = [tf.keras.metrics.sparse_top_k_categorical_accuracy(tf.reshape(a[i],(-1,1))[:,0],
                                                               b[i][:,0], k = 10) for i in range(len(a))]
        return tf.reduce_mean(c)
    
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
  
    return tf.reduce_mean(loss_)

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
                    name='train_accuracy')

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
gpt_decoder (GPTDecoder)     ((None, None, 512), {'dec 42409472  
_________________________________________________________________
dense_120 (Dense)            (None, None, 33547)       17209611  
Total params: 59,619,083
Trainable params: 59,619,083
Non-trainable params: 0
_________________________________________________________________


In [6]:
ep_save = 2
batch_save = 29
model.load_weights('./checkpoints/IFart_epoch_'+str(ep_save)+'_batch_'+str(batch_save)+'/checkpoint.h5py')

In [ ]:
ep_save = 5
batch_save = 13
model.load_weights('./checkpoints/IFart_epoch_'+str(ep_save)+'_batch_'+str(batch_save)+'/checkpoint.h5py')

EPOCHS = 6
# ep_save = 5
for i in range(ep_save, EPOCHS):
    print('**********************      EPOCH '+str(i)+'        **************************')
    
    learning_rate = CustomSchedule(512, factor = 1)
    loss_classif     =  loss_function
    optimizer = tf.keras.optimizers.Adam(
                    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
    metrics_classif  =  [train_accuracy, sparse_k_acc]
    
    model.compile(loss=loss_classif,
              optimizer=optimizer,
              metrics=metrics_classif)
    
    lens = [32, 64, 128, 256, 350, 512]
    bs = [256, 128, 64, 32, 20, 8]
    e = [20,20,30,40,50,80]
    start = batch_save+1
#     start = 1

    for batch in range(start,e[i]):
        print('*********** BATCH '+str(batch))
        
        X = load('batch_'+str(batch), 'batch_tokenized')
#         y = np.concatenate([X[:,1:], np.zeros((X.shape[0], 1))], axis = 1)
    
        max_len = lens[i]
        
        X = X[(X != 0).sum(axis = 1) >= max_len-int(max_len/10)]
        
        X = X[:, :max_len]
        X[X[:,-1] != 0, -1] = 2
        y = np.concatenate([X[:,1:], np.zeros((X.shape[0], 1))], axis = 1)
        
        print('batch_loaded')
        
        batch_size = bs[i]
        epochs = 1
        history = model.fit(X, y, batch_size=batch_size, epochs=epochs)
        
        os.mkdir('./checkpoints/IFart_epoch_'+str(i)+'_batch_'+str(batch))
        model.save_weights('./checkpoints/IFart_epoch_'+str(i)+'_batch_'+str(batch)+'/checkpoint.h5py')

In [ ]:
a = [1,2,3,4,5]

In [7]:
def clean(x):
    import unidecode
    x = str(x)
    x = x.lower()
    x = " ".join(x.split())
    x = unidecode.unidecode(x)
    x = x.replace('\n', '')
    x = '[CLS] ' + x 
    return x

In [43]:
inp = "j'ai toujours eu envie de"
# inp = 'i always thought that'
inp = clean(inp)

# inp1 = "j'ai toujours eu envie de"
# inp1 = clean(inp1)

inp_seq = tokenizer.texts_to_sequences([inp])

In [44]:
inp_seq

[[2, 73, 112, 237, 614, 4]]

In [45]:
from importlib import reload
import generation
reload(generation)

<module 'generation' from 'D:\\fr_dedup.txt\\generation.py'>

In [46]:
a = generation.generate(
        input_ids=inp_seq,
        model = model, 
        max_length=60,
        do_sample=True,
        num_beams=1,
        temperature=0.7,
        top_k=5,
        top_p=0.3,
        repetition_penalty=4,
        bos_token_id=2,
        pad_token_id=0,
        eos_token_ids=3,
        unk_token_ids = 1,
        length_penalty=1,
        vocab_size = 33547,
        remove_unk_tokens = True,
        num_return_sequences = 5
                                )

In [47]:
tokenizer.sequences_to_texts(a)

["[CLS] j'ai toujours eu envie de vous parler des choses que je ne connais pas, mais il faut savoir qu'il y a beaucoup d'autres personnes qui ont un peu plus ou meme une fois par jour pour se rendre compte qu'ils sont en train d'etre en retard et que les gens qui ne savent pas trop comment ils vont les",
 "[CLS] j'ai toujours eu envie de vous parler des choses que je ne sais pas encore comment faire pour me rendre compte du pourquoi et si ca est vrai, c'est parce qu'on a fait un grand effort sur le plan technique mais aussi en tant qu'il y aura bien d'autres choses. il faut savoir ce qui se passe dans les",
 "[CLS] j'ai toujours eu envie de faire un petit tour sur le site et je me suis fait plaisir, en effet, a la fin du mois d'octobre, pour une fois que vous avez bien compris ce qui est arrive au debut des annees 90, mais pas n'importe quoi ! c'est d'ailleurs avec les memes raisons : il y avait deja",
 "[CLS] j'ai toujours eu envie de vous parler des choses que je ne peux pas dire a m